# Preparativos

In [53]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download zaheenhamidani/ultimate-spotify-tracks-db

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [54]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

urlTrain  = 'https://raw.githubusercontent.com/CyberJuan55/Proyecto-IA/master/DataSet/SpotifyFeatures_train.csv'
urlFull = 'https://raw.githubusercontent.com/CyberJuan55/Proyecto-IA/master/DataSet/SpotifyFeatures.csv'
# el train que nos dieron los profes
#df_train_original= pd.read_csv('/content/SpotifyFeatures_train.csv')
df_train_original = pd.read_csv(urlTrain)
df_train = df_train_original.copy()
# el dataset nuestro
df_full_original = pd.read_csv(urlFull)
df_full = df_full_original.copy()


# Exploracion de datos

In [55]:
df_train.head()

,Unnamed: 0,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,107360,Opera,Richard Wagner,Gotterdammerung (Twilight of the Gods): Prolog...,32pnRCr5u3nzO55MgoEJwV,6,0.89900,0.147,131787,0.016,0.947000,B,0.0889,-24.317,Major,0.0380,137.591,4/4,0.0292
1,165017,Reggaeton,Ronald El Killa,Esta Pena,3uMQ7q5oU0VgYl9SyCF2MR,27,0.00613,0.857,255134,0.599,0.000085,B,0.2090,-5.677,Major,0.0951,96.022,4/4,0.5860
2,141044,Indie,LP,Recovery,1a5P7IvLUhRhLnnbkev3PI,59,0.27400,0.518,234905,0.545,0.000023,A#,0.1830,-7.893,Major,0.0385,75.060,4/4,0.1620
3,141709,Indie,The National,Carin at the Liquor Store,4maI5FwVIzjrErduPjjZLO,53,0.59400,0.540,213733,0.615,0.803000,A#,0.1200,-9.264,Major,0.0340,93.170,3/4,0.2390
4,106321,Opera,Renata Tebaldi,Giordano: Andrea Chenier: Come un bel di di ma...,5VEVnqQu6cFewcbWfbkUeE,8,0.95500,0.363,170240,0.196,0.000115,C#,0.6840,-15.675,Minor,0.0452,63.186,4/4,0.2510


In [56]:
#df_full.head()

In [57]:
df_train.columns

Index(['Unnamed: 0', 'genre', 'artist_name', 'track_name', 'track_id',
       'popularity', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

In [58]:
df_train.dtypes

Unnamed: 0            int64
genre                object
artist_name          object
track_name           object
track_id             object
popularity            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo               float64
time_signature       object
valence             float64
dtype: object

# Limpieza de datos

In [59]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)


In [60]:

indexNames = df_train[ df_train['genre'] == 'Comedy' ].index
df_train.drop(indexNames , inplace=True)

In [61]:
df_train['genre'].value_counts()

Soundtrack          7758
Jazz                7634
Indie               7603
Children’s Music    7554
Pop                 7532
Electronic          7500
Rock                7455
Hip-Hop             7422
Folk                7406
Classical           7395
Rap                 7367
Alternative         7342
Soul                7280
R&B                 7242
World               7211
Blues               7187
Ska                 7154
Reggaeton           7139
Anime               7131
Reggae              7019
Dance               6950
Country             6861
Opera               6612
Movie               6261
Children's Music    4298
A Capella             96
Name: genre, dtype: int64

Vemos que la columna Children esta repetida

In [62]:
indexNames = df_train[ df_train['genre'] == "Children's Music" ].index
df_train.drop(indexNames , inplace=True)

### Busqueda de datos repetidos

In [107]:

if df_train.duplicated().sum() == 0:
  print('no hay datos duplicados')
else:
  df_train.drop_duplicates


no hay datos duplicados


### Busqueda de datos faltantes

In [64]:
#columnas con datos faltantes
cols_with_missing = [col for col in df_train.columns if df_train[col].isnull().any()] 
faltanDatos = True
if len(cols_with_missing)==0:
  faltanDatos= False
  print('no faltan valores') 

no faltan valores


In [65]:
# saco las filas correspondientes
if faltanDatos == True:
    print(cols_with_missing)
    df_train.dropna(axis=0, inplace=True)


# Variables Categoricas

In [66]:
# Variable categoricas
s = (df_train.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

['genre', 'artist_name', 'track_name', 'track_id', 'key', 'mode', 'time_signature']


In [67]:
list_of_timeSignature = df_train['time_signature'].unique()
print(list_of_timeSignature)

['4/4' '3/4' '5/4' '1/4' '0/4']


In [68]:
df_train['time_signature'].value_counts()

4/4    153286
3/4     15927
5/4      3294
1/4      1598
0/4         6
Name: time_signature, dtype: int64

Decidimos no utilizar Time_signature como feature por varios motivos


*   En musica no existe compaces de 1/4 ni de 0/4 entre ambos suman 1656 valores lo que nos parecio un numero considerable
*   En una cancion puede haber multiples cambios de compaces, en la musica progresiva es algo muy comun

*   La mayoria de la musica se encuentra en 4/4 podemos ver que hay 156851 valores que considerablemente mayor a lo suma de todas las otras categorias esto introduceria un gran sesgo en nuestro modelo 










In [69]:
df_train = df_train.drop(['time_signature'], axis=1)

In [70]:
# variables categoricas con baja cardinalidad
categorical_cols = [cname for cname in df_train.columns if
                    df_train[cname].nunique() < 14 and 
                    df_train[cname].dtype == "object"]
print(categorical_cols)

['key', 'mode']


Quedan descartadas las variables categoricas: 

*   Time_signature por los motivos comentados anteriormente
*   track_id: no hay ninguna razon para utilizarla
*   el nombre del artista: son demasiadas variables de momento las sacamos pero es cierto que es un feature interesente 
*   track_name: todos sus valores son distintos 







In [71]:
list_of_modes = df_train['mode'].unique()
print(list_of_modes)

['Major' 'Minor']


In [72]:
list_of_keys = df_train['key'].unique()
print(list_of_keys)


['B' 'A#' 'C#' 'C' 'F#' 'E' 'A' 'D' 'G' 'F' 'G#' 'D#']


# Removiendo datos de columnas faltantes, separacion en entrenamiento y validacion

In [73]:
# Remove rows with missing target, separate target from predictors

X = df_train.drop(columns=['genre'])
y = df_train['genre']


In [74]:
from sklearn.model_selection import train_test_split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(df_train, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [75]:
y_train.value_counts().sort_index()

A Capella             74
Alternative         5813
Anime               5700
Blues               5718
Children’s Music    6002
Classical           5947
Country             5409
Dance               5612
Electronic          6063
Folk                5997
Hip-Hop             5885
Indie               6121
Jazz                6117
Movie               5032
Opera               5283
Pop                 6012
R&B                 5839
Rap                 5856
Reggae              5608
Reggaeton           5707
Rock                5937
Ska                 5777
Soul                5824
Soundtrack          6184
World               5771
Name: genre, dtype: int64

In [76]:
y_valid.value_counts().sort_index()

A Capella             22
Alternative         1529
Anime               1431
Blues               1469
Children’s Music    1552
Classical           1448
Country             1452
Dance               1338
Electronic          1437
Folk                1409
Hip-Hop             1537
Indie               1482
Jazz                1517
Movie               1229
Opera               1329
Pop                 1520
R&B                 1403
Rap                 1511
Reggae              1411
Reggaeton           1432
Rock                1518
Ska                 1377
Soul                1456
Soundtrack          1574
World               1440
Name: genre, dtype: int64

In [77]:

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

print(numerical_cols)

['popularity', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']


In [78]:
#Seleccion de columnas
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()


# Un poco de ingenieria de datos

In [79]:
from sklearn.preprocessing import OrdinalEncoder


label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

ordinal_encoder = OrdinalEncoder()
label_X_train[categorical_cols] = ordinal_encoder.fit_transform(X_train[categorical_cols])
label_X_valid[categorical_cols] = ordinal_encoder.transform(X_valid[categorical_cols])




### Viendo si la transformacion esta bien

In [80]:
label_X_valid.head()

,key,mode,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
170785,3.0,0.0,53,0.000018,0.389,251867,0.925,0.244000,0.2220,-7.970,0.0643,150.893,0.318
157944,4.0,0.0,45,0.098300,0.746,242000,0.575,0.309000,0.3450,-9.032,0.0267,119.698,0.591
56031,10.0,0.0,47,0.646000,0.513,195147,0.818,0.013600,0.1830,-6.770,0.0425,79.440,0.902
71230,0.0,0.0,0,0.006120,0.721,226010,0.724,0.000004,0.0541,-7.077,0.0689,107.770,0.651
130337,7.0,0.0,49,0.603000,0.693,294920,0.437,0.398000,0.1680,-8.472,0.0336,131.000,0.462


In [81]:
X_valid.head()

,key,mode,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
170785,C,Major,53,0.000018,0.389,251867,0.925,0.244000,0.2220,-7.970,0.0643,150.893,0.318
157944,C#,Major,45,0.098300,0.746,242000,0.575,0.309000,0.3450,-9.032,0.0267,119.698,0.591
56031,G,Major,47,0.646000,0.513,195147,0.818,0.013600,0.1830,-6.770,0.0425,79.440,0.902
71230,A,Major,0,0.006120,0.721,226010,0.724,0.000004,0.0541,-7.077,0.0689,107.770,0.651
130337,E,Major,49,0.603000,0.693,294920,0.437,0.398000,0.1680,-8.472,0.0336,131.000,0.462


In [82]:
label_X_train.head()

,key,mode,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
101822,1.0,0.0,10,0.87200,0.363,52067,0.135,0.96700,0.4640,-23.681,0.0390,84.979,0.400
83812,5.0,0.0,28,0.49300,0.803,172067,0.358,0.00474,0.0991,-16.478,0.1690,133.776,0.916
2556,4.0,0.0,44,0.73400,0.352,667745,0.357,0.86400,0.1360,-10.013,0.0334,130.184,0.213
174265,11.0,1.0,48,0.21200,0.631,266667,0.793,0.00000,0.0441,-7.551,0.2510,175.814,0.826
136222,9.0,1.0,68,0.00163,0.504,214520,0.891,0.00199,0.1480,-5.632,0.1370,153.988,0.628


In [83]:
X_train.head()

,key,mode,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
101822,A#,Major,10,0.87200,0.363,52067,0.135,0.96700,0.4640,-23.681,0.0390,84.979,0.400
83812,D,Major,28,0.49300,0.803,172067,0.358,0.00474,0.0991,-16.478,0.1690,133.776,0.916
2556,C#,Major,44,0.73400,0.352,667745,0.357,0.86400,0.1360,-10.013,0.0334,130.184,0.213
174265,G#,Minor,48,0.21200,0.631,266667,0.793,0.00000,0.0441,-7.551,0.2510,175.814,0.826
136222,F#,Minor,68,0.00163,0.504,214520,0.891,0.00199,0.1480,-5.632,0.1370,153.988,0.628


# Probando modelos


###  Regresion Logistica














In [84]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class = 'multinomial', solver='lbfgs', max_iter=200)

model.fit(label_X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [102]:
y_pred = model.predict(label_X_valid)

In [103]:
from sklearn.metrics import classification_report, accuracy_score
print(accuracy_score(y_valid, y_pred))

0.11549837750911754


### Random Forest

In [104]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=30, random_state=42)
rfc.fit(label_X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [105]:
predict = rfc.predict(label_X_valid)
print(accuracy_score(y_valid, predict))


0.33348648881486376


### GridSerchCv en RandonForest

In [89]:
# Numero de arboles
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Numero de features considerado al dividir
max_features = ['auto', 'sqrt']
# numero maximo de niveles
max_depth = [2,4]
# Numero minimo de niveles por nodo
min_samples_split = [2, 5]
# Numero minimo de niveles por hoja
min_samples_leaf = [1, 2]
# metodo de seleccion por arbol 
bootstrap = [True, False]

In [90]:
# Grilla
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [91]:
from sklearn.model_selection import GridSearchCV
rfc_Grid = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [92]:
#rfc_Grid.fit(label_X_train, y_train)


In [93]:
#rfc_Grid.best_params_

In [94]:

#clf  = RandomForestClassifier(n_estimators=80, random_state=42,bootstrap=True,max_depth= 4, max_features='auto',min_samples_leaf=1,min_samples_split=2)
#clf.fit(label_X_train, y_train)
#y_clf = clf.predict(label_X_valid)


In [95]:
#print(accuracy_score(y_valid, y_clf))


In [96]:
#from sklearn.externals import joblib
#joblib.dump(rfc_Grid,'modelo_entrenado.pkl')